In [ ]:
import pandas as pd
import numpy as np
#import timeit

In [ ]:
'''Dictionary based on category column from main database'''

categories_selected = {

        'coal':
            {'hard_coal':
                 {'anthracite':
                      [0],
                 'bituminous_coal':
                      ['coking_coal','other_bituminous_coal']},
             'brown_coal':
                 {'sub_bituminous_coal':
                      [0],
                 'lignite':
                      [0]}, 
             'coal_products':
                 {'coal_coke':
                      ['coke_oven_coke','gas_coke'],
                 'patent_fuel':
                      [0],
                 'brown_coal_briquettes':
                      [0],
                 'coal_tar':
                      [0],
                 'gasworks_gas':
                      ['blast_furnace_gas','other_recovered_gases'],
                 'other_coal_products':
                      [0]}},           
        'peat_and_peat_products':
            {'peat':
                 {0:
                      [0]},
             'peat_products':
                 {0:
                      [0]}}, 
        'oil_shale_oil_sands':
            {0:
                 {0:
                     [0]}},
        'natural_gas_including_lng':
            {0:
                 {0:
                     [0]}},
        'oil':
            {'conventional_crude_oil':
                 {0:
                      [0]},
             'natural_gas_liquids':
                 {0:
                      [0]},
             'refinery_feedstocks':
                 {0:
                      [0]},
             'additives_and_oxygenates':
                 {0:
                      [0]},
             'other_hydrocarbons':
                 {0:
                      [0]},
             'oil_products':
                 {'refinery_gas':
                      [0],
                 'ethane':
                      [0],
                 'liquified_petroleum_gas':
                      [0],
                 'naphtha':
                      [0],
                 'gasolines':
                      ['aviation_gasoline','motor_gasoline','gasoline_type_jet_fuel'],
                 'kerosenes':
                      ['kerosene_type_jet_fuel','other_kerosene'],
                 'gas_oil_diesel_oil_and_heavy_gas_oil':
                      ['gas_oil_diesel_oil',],
                 'fuel_oil':
                      [0],
                 'other_oil_products':
                      ['white_spirit_and_special_boiling_point_industrial_spirits','lubricants',
                                      'paraffin_waxes','petroleum_coke','bitumen','other_oil_products_n_e_c']}}, 
        'biofuels':
            {'solid_biofuels':
                 {'fuelwood':
                      [0],
                  'bagasse':
                      [0],
                  'animal_waste':
                      [0],
                  'black_liquor':
                      [0],
                  'charcoal':
                      [0],
                  'other_vegetal_material_and_residues':
                      [0]},
            'liquid_biofuels':
                 {'biodiesel':
                      [0],
                  'biogasoline':
                      [0],
                  'other_liquid_biofuels':
                      [0]},
            'biogases':
                 {0:
                      [0]}},
        'waste':
            {'industrial_waste':
                 {0:
                      [0]},
             'municipal_wastes':
                 {0:
                      [0]}},
        'electricity': 
            {'electricity_net_installed_capacity_of_electric_power_plants':
                 {0:
                      [0]}},
        'heat':
            {0:
                 {0:
                      [0]}},
        'nuclear_fuels':
            {'uranium':
                 {0:
                      [0]}}}

In [ ]:
#Utworzenie listy na podstawie słownika
c_list=list()
for i,j in categories_selected.items():
    #print('L1',i)
    for ii,jj in j.items():
        #print('L2',ii)
        for iii,jjj in jj.items():
            for x in jjj:
                c_list.append([i,ii,iii,x])
                print('L1',i,'L2',ii,'L3',iii, 'L4',x,)



In [ ]:
#przypisanie nagłóków do listy
lts = pd.DataFrame(c_list)
lts.columns = ['level_1','level_2', 'level_3', 'level_4']
lts

In [ ]:
#wczytanie bazy danych z pliku redy_data
df = pd.read_csv('transaction_type021.csv',low_memory=False) 

In [ ]:
df.sample(10)

In [ ]:
#utworzenie nowych kolumn
df['L1'] = 0
df['L2'] = 0
df['L3'] = 0
df['L4'] = 0

In [ ]:
#podgląd
df.sample(10)

## Numpy

In [ ]:
#from data frame to numpy array (faster than df)
lts_np = lts.to_numpy(dtype=str)
ctg_np = df.to_numpy(dtype=str)

In [ ]:
s = 0 #pierwszy wiersz
e = len(ctg_np) #ostatni wiersz  
#Pętla iteruje kolejno przez każdy wiersz i sprawdza czy dana kategoria istnieje w słowniku, jeśli tak
#to uzupełnia tablicę w utworzonych we wcześniejszym etapie kolumnach L1,L2,L3,L4
for x in range(s,e): 
    if ctg_np[x][6] in lts_np: #warunek sprawdza czy dana kategori znajduje się w słowniku
        mask = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] #początkowa maska
        #jak już wadomo, że dana kategoria jest w słowniku to określane są jej 'współrzędne w słowniku, czyli i-wiersz,j-kolumna 
        i,j = np.where(lts_np == ctg_np[x][6]) 
        ipt = lts_np[i[0]][j[0]] #input do funkcji place, czyli wybierana jest kategoria ze słownika 
        dest = ctg_np[x] #docelowa tablica 
        mask[11+j[0]+1] = 1 # maska, czyli gdzie ma być dodana kategoria w tablicy docelowej
        np.place(dest,mask,ipt) #funkcja, która dodaje kategorie ze słownika do docelowej tablicy
        #Jeśli dana kategoria jest np. na poziomie 4 to trzeba uzupełnić wyższe poziomy na podstawie słownika
        if j[0] > 0: #warunek sprawdzający czy dana kategoria jest na poziomie niższym niż 1 ('L1'). Jeśłi jest na L2 to uzupełnia L1, itd.
            for xx in range(0,j[0]):
                mask = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                mask[11+j[0]-xx] = 1
                ipt = lts_np[i[0]][j[0]-1-xx]
                np.place(dest,mask,ipt) 
  

In [ ]:
# konwersja z powrotem do data frame
df = pd.DataFrame(ctg_np)
df.columns = ['country_or_area','ISO','year','transaction_type',
              'unit','quantity','category','CL4','CL3','CL2',
              'commodity_transaction','commodity','L1','L2','L3','L4']

In [ ]:
df.sample(10)

## Units

In [ ]:
df.unit.unique()

In [ ]:
un = {
    'Metric tons,  thousand':
        {1:'Metric tons,  thousand'},
    'Terajoules':
        {1*10**12:'Kilowatts,  thousand'},
    'Kilowatts,  thousand':
        {1:'Kilowatts,  thousand'},
    'Kilowatt-hours, million':
        {1000:'Kilowatts,  thousand'},
    'Cubic metres, thousand':
        {1:'Cubic metres, thousand'},
    'Metric Tons':
        {1/1000:'Metric tons,  thousand'}
    }

In [ ]:
new_quantity_column = list()
new_unit_column = list()
size = df.L1.size

for x in range(0,size):
    v = list(un[df.unit.iloc[x]].values())
    k = list(un[df.unit.iloc[x]].keys())
    if df.unit.iloc[x] == v[0]:
        aa = k[0]
        new_unit = v[0]
        new_quantity = float(df.quantity.iloc[x]) * aa
        new_quantity_column.append(new_quantity)
        new_unit_column.append(new_unit)
    elif df.unit.iloc[x] == 'Terajoules':
        new_quantity = float(df.quantity.iloc[x])*(277777.778) #terajoule to kilowatt hour
        new_unit = v[0]
        new_quantity_column.append(new_quantity)
        new_unit_column.append(new_unit)
    else:
        aa = k[0]
        new_quantity = float(df.quantity.iloc[x]) * aa
        new_unit = v[0]
        new_quantity_column.append(new_quantity)
        new_unit_column.append(new_unit)

In [ ]:
new_columns = {'new_unit':new_unit_column, 'new_quantity':new_quantity_column}
dff = pd.DataFrame(new_columns)

In [ ]:
df = df.join(dff)

In [ ]:
df.sample(10)

In [ ]:
df.to_csv('ready_data_030.csv', index=False)